# BERT Email Classification on Google Colab

This notebook fine-tunes a DistilBERT model to classify emails into **Complaint**, **Request**, **Feedback**, or **Spam**.

**Instructions:**
1. Run the installation cell.
2. Upload your `labeled_email_dataset.csv` when prompted.
3. Run the training cells.
4. Download the trained model if needed.

In [ ]:
# Install dependencies
!pip install transformers accelerate datasets scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
from google.colab import files
import io

In [ ]:
# Upload Dataset
print("Please upload 'labeled_email_dataset.csv'...")
uploaded = files.upload()

In [ ]:
# Load Data
filename = list(uploaded.keys())[0]
df = pd.read_csv(io.BytesIO(uploaded[filename]))
df = df.dropna(subset=['body', 'category'])

# Create Label Mapping
label_map = {label: i for i, label in enumerate(df['category'].unique())}
print(f"Label Mapping: {label_map}")
df['label'] = df['category'].map(label_map)

# Split Data
X_train, X_test, y_train, y_test = train_test_split(df['body'], df['label'], test_size=0.2, random_state=42, stratify=df['label'])

In [ ]:
# Dataset Class
class EmailDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
# Initialize Tokenizer using Hugging Face
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Create Datasets
train_dataset = EmailDataset(X_train.to_numpy(), y_train.to_numpy(), tokenizer)
test_dataset = EmailDataset(X_test.to_numpy(), y_test.to_numpy(), tokenizer)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
# Initialize Model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(label_map))

# Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,              # Increased for better results on GPU
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
# Train
trainer.train()

In [ ]:
# Evaluate
trainer.evaluate()

In [ ]:
# Save Model
model.save_pretrained('./bert_email_classifier')
tokenizer.save_pretrained('./bert_email_classifier')

# Download Model (Zipped)
!zip -r bert_email_classifier.zip ./bert_email_classifier
files.download('bert_email_classifier.zip')